<a href="https://colab.research.google.com/github/jiya2107/Machine-Learning---Practical/blob/main/ML_Experiment_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import numpy as np

# Load and preprocess data (same as experiment 3)
try:
    df = pd.read_csv('HepatitisCdata.csv', index_col=0)
except FileNotFoundError:
    print("Please ensure 'HepatitisCdata.csv' is uploaded.")
    exit()

df = df.replace('?', np.nan)
for col in ['ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']:
    df[col] = pd.to_numeric(df[col])
df = df.dropna()

df['Sex'] = df['Sex'].map({'m': 1, 'f': 0})
# Handle '0s=suspect Blood Donor' by replacing '0s' with '0' before splitting
df['Category'] = df['Category'].str.replace('0s=', '0=').str.split('=').str[0].astype(int)

X = df.drop('Category', axis=1)
y = df['Category']

# Split and Scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- 1. LDA as a Classifier ---
lda_classifier = LinearDiscriminantAnalysis()
lda_classifier.fit(X_train_scaled, y_train)

y_pred = lda_classifier.predict(X_test_scaled)

print("--- Linear Discriminant Analysis (LDA) as Classifier Results ---")
print("Classification Report:")
print(classification_report(y_test, y_pred))

# --- 2. LDA for Dimensionality Reduction (Max components = C-1) ---
# C=4 classes after preprocessing -> max 3 components
lda_reduction = LinearDiscriminantAnalysis(n_components=3)
lda_reduction.fit(X_train_scaled, y_train)
X_train_lda = lda_reduction.transform(X_train_scaled)

print("\n--- LDA for Dimensionality Reduction Results ---")
print(f"Original Feature Count: {X_train_scaled.shape[1]}")
print(f"Reduced LDA Shape: {X_train_lda.shape}")
print(f"Proportion of variance explained by each discriminant function:")
print(lda_reduction.explained_variance_ratio_.round(4))